In [3]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
import time
from IPython.display import clear_output

In [4]:
rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓


⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [5]:
def QLearning_Off_Policy(gridworld:GridWorld_v2.GridWorld_v2,TrajectoryPoolSize = 50,gamma = 0.99,trajectorySteps=-1, learning_rate=0.001, num_episodes=600)->GridWorld_v2.GridWorld_v2:
    """
    这是最基础的SARSA算法

    Parameters:
    trajectorySteps (int): 寻路的轨迹长度，如果是-1，则为寻到目的则停止，否则参数即为trajectory长度
    learning_rate (float): 学习率，用于调节TD-target
    epsilon (float): epsilon-greedy的核心参数，0~1的浮点数，其中1则表示当前state所有决策概率一样，0则表示决策没有任何的随机性
    num_episodes (int): 表示模型迭代次数

    Returns:
    GridWorld_v2.GridWorld_v2: 把模型返回回去
    """
    
    state_value = np.zeros((rows * columns))
    action_value = np.zeros((rows * columns, 5))
    policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
    # behavior_policy = 1000
    behavior_policy = np.ones((rows * columns, 5)) * 0.2
    epsilon = 0.5
    
    initState=10

    TrajectoryPool = []
    TrajectoryPoolSize = 50
    for i in range(TrajectoryPoolSize):
        if trajectorySteps==-1:
            stop_when_reach_target = True
            # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
            # 这里改成behavior_policy
            Trajectory = gridworld.getTrajectoryScore(nowState=initState, 
                                                      action=random.randint(0,4) , 
                                                      policy=behavior_policy, 
                                                      steps=trajectorySteps, 
                                                      stop_when_reach_target=True)
        TrajectoryPool.append(Trajectory)
        
    for episode in range(num_episodes):
        if episode
        #清除输出，可以更好的展示策略
        time.sleep(0.2)
        clear_output(wait=True)
        print("episode",f"{episode}/{num_episodes}")
        
        #cnt数组用来检查每个state有多少次访问
        cnt = [0 for i in range(25)]
        
        for Trajectory in TrajectoryPool:
            steps = len(Trajectory) - 1
            for k in range(steps,-1,-1):
                #State，Action，Reward，NextState，NextAction
                tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
                cnt[tmpstate] += 1
                #Q.learning
                TD_error = action_value[tmpstate][tmpaction] - (tmpscore + gamma * action_value[nextState].max())
                action_value[tmpstate][tmpaction] -= learning_rate * TD_error

        # policy improvement
        policy = np.eye(5)[np.argmax(action_value,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
    
        #输出每个state的访问次数
        print(np.array(cnt).reshape(5,5))

        state_value = np.sum(policy * action_value,axis=1)
        print(state_value)
        mean_state_value = np.sum(policy * action_value,axis=1).mean()
        
        gridworld.showPolicy(policy)
        print(np.round(state_value,decimals=4).reshape(5,5))
        print("mean_state_value", mean_state_value)

    return action_value
    

In [7]:
action_value = QLearning_Off_Policy(gridworld)

episode 599/600
[[173 155  96  78  44]
 [165 137 104  83  70]
 [222 130  77  75  68]
 [176  97  50  60  73]
 [154  90  56  56  66]]
[0.28621912 0.3571427  0.45624884 0.64091655 0.87661114 0.23123789
 0.27681206 0.69890607 0.899957   1.20883866 0.19100077 0.13740874
 5.70390861 1.13309322 1.60495566 0.14503632 5.67318974 5.25681493
 4.98692532 2.34908835 0.10913264 4.8536694  5.35316034 4.44879511
 3.06475208]
➡️➡️➡️⬇️⬇️
⬆️⏫️⏩️⬇️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
[[0.2862 0.3571 0.4562 0.6409 0.8766]
 [0.2312 0.2768 0.6989 0.9    1.2088]
 [0.191  0.1374 5.7039 1.1331 1.605 ]
 [0.145  5.6732 5.2568 4.9869 2.3491]
 [0.1091 4.8537 5.3532 4.4488 3.0648]]
mean_state_value 2.0377528510777583


In [13]:
import pickle
with open('true_action_value.pkl','wb') as file:
    pickle.dump(action_value,file)